# 付録_プログラミングと数学との橋渡し
ここでは、本書全体で扱った数学の基礎を学ぶうえで必要なプログラムを実行していく流れを学んでいきます。 

In [ ]:
#Colaboratory環境の設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Appendix

In [ ]:
#ライブラリの設定
!pip install -q -r ./requirements.txt

## A-1 数式を動かして理解する正規分布

### 確率密度関数の定義

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

# 正規分布を定義
def normal_distribution(x,mu,sigma):
    y = 1/np.sqrt(2*np.pi*sigma**2)*np.exp(-(x-mu)**2/(2*sigma**2))
    return y

### 確率密度関数の描画

In [ ]:
# 正規分布のパラメータ設定
mu = 116.6
sigma = 4.8

# 描画パラメータ設定
x_min = 80
x_max = 150
x_num = 100

# 正規分布の計算
x = np.linspace(x_min, x_max, x_num)
y = normal_distribution(x,mu,sigma)

# 正規分布の描画
plt.plot(x, y ,color="k")
plt.show()
%matplotlib inline

### 確率密度関数の積分による面積の導出

In [ ]:
# 積分範囲の設定
x_min = 115
x_max = 117
x_num = 100

# 積分範囲における正規分布の値の計算
x = np.linspace(x_min, x_max, x_num)
y = normal_distribution(x,mu,sigma)

# 積分の計算
dx = (x_max-x_min)/(x_num-1)
prob = 0
for i in range(x_num):
    y = normal_distribution(x[i],mu,sigma)
    prob += y*dx
print("確率:",prob)

## A-2 微分方程式の差分化による誤差とテイラー展開

In [ ]:
from sympy import*
import numpy as np
from matplotlib import pyplot as plt

# パラメータを設定
n = 1     # 次数
x0 = 0    # 初期値

# シンボルを定義
x = Symbol('x')

# 関数を定義
f = 2 + x + sin(x) + exp(x)/10

# テイラー展開を導出
taylor = series(f, x=x, x0=x0, n=n+1).removeO()
taylor_y = lambdify(x, taylor, 'numpy') 
print("テイラー展開")
print(taylor)

# 描画
x_theory = np.arange(0.0, 10.0, 0.1)
y_theory = 2+x_theory+np.sin(x_theory)+np.exp(x_theory)/10
plt.plot(x_theory, y_theory, lw=3, c="k")
plt.plot(x_theory, taylor_y(x_theory),c="b")
plt.xlim([0,10])

## 11-3 非線形最適化としての機械学習/深層学習における回帰/分類

### （非線形最適化）関数の定義

In [ ]:
def function(x):
    y = x**2
    return y

def differential(x,dx):
    dy = (function(x+dx)-function(x))/dx
    return dy

### （非線形最適化）最急降下法の実行

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

# 関数生成
x_list = np.arange(-10, 11)
y_list = function(x_list)
num = len(x_list)

# パラメータ設定
dx = 0.1     # 刻み幅（学習率）
iter = 200   # 繰り返し回数

# 初期値設定
x = -10

# 繰り返し処理
list_plot = []
fig = plt.figure()
for t in range(iter):
    # 導関数を導出
    dy = differential(x,dx)
    # x,yを更新
    x = x - np.sign(dy)*dx
    y = function(x)
    # グラフを描画
    img = plt.plot(x,y,marker='.', color="red",markersize=20)
    img += plt.plot(x_list,y_list,color="black")
    list_plot.append(img)

# グラフ（アニメーション）描画
plt.grid()
anim = animation.ArtistAnimation(fig, list_plot, interval=200, repeat_delay=100)
rc('animation', html='jshtml')
plt.close()
anim

### （非線形最適化）処理部のみ実行

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# パラメータ設定
delta = 0.01     # 刻み幅（学習率）
iter = 200       # 繰り返し回数

# 初期値設定
x = -10

# 繰り返し処理
list_plot = []
series_y = []
fig = plt.figure()
for t in range(iter):
    # 導関数を導出
    dy = differential(x,dx)
    # x,y を更新
    x = x - delta*dy
    y = function(x)
    series_y.append(y)
    
# グラフを描画
plt.plot(series_y,c="k")

### （回帰分析）関数の定義

In [ ]:
def function(X,y,alpha,beta):
    cost = (1/(2*m))*np.sum((beta+alpha*X-y)**2)
    return cost

def differential_alpha(X,y,alpha,beta,delta):
    d_cost = (function(X,y,alpha+delta,beta)-function(X,y,alpha,beta))/delta
    return d_cost

def differential_beta(X,y,alpha,beta,delta):
    d_cost = (function(X,y,alpha,beta+delta)-function(X,y,alpha,beta))/delta
    return d_cost

### （回帰分析）データ読み込み

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# データ読み込み
df_sample = pd.read_csv("sample_linear.csv")
sample = df_sample.values.T

# 変数を設定
X = sample[0]
y = sample[1]

### （回帰分析）最急降下法の実行

In [ ]:
# パラメータ設定
delta = 0.001     # 刻み幅（学習率）
iter = 20000      # 繰り返し回数

# 初期値設定
alpha = 1
beta = 1

# 繰り返し処理
cost = np.zeros(iter)
da = np.zeros(iter)
m = len(y)
for i in range(iter):
    
    # 導関数を導出
    d_alpha = differential_alpha(X,y,alpha,beta,delta)
    d_beta = differential_beta(X,y,alpha,beta,delta)
    
    # alpha, beta, costを更新
    alpha = alpha - delta*d_alpha
    beta = beta - delta*d_beta
    cost[i] = function(X,y,alpha,beta)
    da[i] = alpha
    
# グラフを描画
plt.plot(da,c="k")

### （回帰分析）最小二乗近似の結果を描画

In [ ]:
plt.scatter(sample[0],sample[1],c="k")
plt.plot(X,beta+alpha*X,color="red")
plt.show()